<a href="https://colab.research.google.com/github/kyungminkim-dev/boostcamp-ai-tech/blob/main/4_fancy_rnn_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**4. LSTM, GRU**
1. 기존 RNN과 다른 부분에 대해서 배웁니다.
2. 이전 실습에 이어 다양한 적용법을 배웁니다.

### **필요 패키지 import**

In [1]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch

### **데이터 전처리**

아래의 sample data를 확인해봅시다.  
이전 실습과 동일합니다.

In [2]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [3]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

100%|██████████| 10/10 [00:00<00:00, 42625.04it/s]

Maximum sequence length: 20


In [4]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

### **LSTM 사용**

LSTM에선 cell state가 추가됩니다.  
Cell state의 shape는 hidden state의 그것과 동일합니다.

In [5]:
embedding_size = 256
hidden_size = 512
num_layers = 1
num_dirs = 1

embedding = nn.Embedding(vocab_size, embedding_size)
lstm = nn.LSTM(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

In [6]:
# d_w: word embedding size
batch_emb = embedding(batch)  # (B, L, d_w)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, (h_n, c_n) = lstm(packed_batch, (h_0, c_0))
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)
print(c_n.shape)

PackedSequence(data=tensor([[-0.1079, -0.0388,  0.1809,  ...,  0.1815, -0.0696, -0.0550],
        [ 0.0073, -0.0524,  0.0064,  ...,  0.0433,  0.0440, -0.1180],
        [ 0.0087, -0.1701,  0.1332,  ...,  0.0424,  0.0783,  0.0684],
        ...,
        [ 0.0535, -0.0929,  0.1132,  ...,  0.0064, -0.0511, -0.1488],
        [-0.0552,  0.1196,  0.0977,  ...,  0.1745,  0.0498,  0.1764],
        [ 0.0041,  0.0678,  0.0419,  ...,  0.0495,  0.1681, -0.0535]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])


In [7]:
outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)
print(output_lens)

torch.Size([20, 10, 512])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


### **GRU 사용**

GRU는 cell state가 없어 RNN과 동일하게 사용 가능합니다.   
GRU를 이용하여 LM task를 수행해봅시다.

In [8]:
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [9]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [10]:
input_id = batch.transpose(0, 1)[0, :]  # (B)
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

Teacher forcing 없이 이전에 얻은 결과를 다음 input으로 이용합니다.

In [11]:
for t in range(max_len):
  input_emb = embedding(input_id).unsqueeze(0)  # (1, B, d_w)
  output, hidden = gru(input_emb, hidden)  # output: (1, B, d_h), hidden: (1, B, d_h)

  # V: vocab size
  output = output_layer(output)  # (1, B, V)
  probs, top_id = torch.max(output, dim=-1)  # probs: (1, B), top_id: (1, B)

  print("*" * 50)
  print(f"Time step: {t}")
  print(output.shape)
  print(probs.shape)
  print(top_id.shape)

  input_id = top_id.squeeze(0)  # (B)

**************************************************
Time step: 0
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 1
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 2
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 3
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 4
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 5
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 6
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 7
torch.Size([1, 10, 100])
torch.Si

`max_len`만큼의 for 문을 돌면서 모든 결과물의 모양을 확인했지만 만약 종료 조건(예를 들어 문장의 끝을 나타내는 end token 등)이 되면 중간에 생성을 그만둘 수도 있습니다.

### **양방향 및 여러 layer 사용**

이번엔 양방향 + 2개 이상의 layer를 쓸 때 얻을 수 있는 결과에 대해 알아봅니다.

In [12]:
num_layers = 2
num_dirs = 2
dropout=0.1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    bidirectional=True if num_dirs > 1 else False
)

Bidirectional이 되었고 layer의 개수가 $2$로 늘었기 때문에 hidden state의 shape도 `(4, B, d_h)`가 됩니다.

In [13]:
# d_w: word embedding size, num_layers: layer의 개수, num_dirs: 방향의 개수
batch_emb = embedding(batch)  # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

PackedSequence(data=tensor([[ 0.0156,  0.0698, -0.1230,  ...,  0.0401,  0.1630, -0.0326],
        [ 0.0930, -0.0770, -0.0298,  ...,  0.2019,  0.1865,  0.1806],
        [ 0.0395,  0.0197, -0.0036,  ..., -0.0224,  0.0104,  0.1032],
        ...,
        [-0.0019, -0.1845, -0.0968,  ..., -0.0524, -0.0444,  0.0231],
        [-0.1041, -0.1197, -0.0872,  ...,  0.0534,  0.1473, -0.0249],
        [-0.0543, -0.1646, -0.1082,  ...,  0.0542,  0.1120, -0.0473]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 1024])
torch.Size([4, 10, 512])


In [14]:
outputs, output_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

torch.Size([20, 10, 1024])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


각각의 결과물의 shape는 다음과 같습니다.

`outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
`h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [15]:
batch_size = h_n.shape[1]
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)

tensor([[[[-0.2235,  0.0167,  0.3821,  ..., -0.0615,  0.3001,  0.0160],
          [-0.1702,  0.1032, -0.2759,  ..., -0.1462, -0.1748, -0.3642],
          [-0.2246, -0.2220, -0.4224,  ...,  0.0091, -0.3579,  0.1926],
          ...,
          [ 0.0811,  0.3725, -0.0195,  ...,  0.3946,  0.1714, -0.2866],
          [-0.2144,  0.0137,  0.1006,  ..., -0.0782, -0.2732,  0.3447],
          [-0.3433,  0.0240, -0.2747,  ...,  0.1367,  0.0552,  0.3710]],

         [[-0.1304,  0.3760, -0.0852,  ...,  0.2363,  0.1978,  0.0600],
          [ 0.0468, -0.3332, -0.0279,  ...,  0.3648,  0.1310, -0.0396],
          [-0.0091,  0.2089,  0.3184,  ...,  0.2013, -0.4742,  0.2713],
          ...,
          [ 0.0871,  0.2624,  0.0079,  ..., -0.1740,  0.0507,  0.2318],
          [ 0.0509,  0.3468, -0.3102,  ..., -0.2001, -0.1578, -0.0049],
          [-0.1052, -0.3954, -0.1468,  ...,  0.0547,  0.2976, -0.0630]]],


        [[[-0.0543, -0.1646, -0.1082,  ..., -0.1417,  0.1295, -0.2944],
          [-0.1276, -0.1257,